In [ ]:
import numpy as np
import glob
import os

def read_flt_traj(*args):
    fName = args[0]         # file dir
    imax=13                 # record size
    ieee='b'                # IEEE big-endian format
    WORDLENGTH = 8          # default 8 bytes per real*8
    prec = np.float64
    if (len(args)==2):
        WORDLENGTH = args[1]
        if WORDLENGTH ==4 :
            prec = np.float32
            
    bytesPerRec=imax*WORDLENGTH;

    fls = sorted(glob.glob( fName + '*.data' ))
    
    header =np.zeros([0,13])
    data   =np.zeros([0,13])
 
    #### Read everything
    for k in range(len(fls)):
        file_length_in_bytes = os.path.getsize(fls[k])
        nrecs = file_length_in_bytes/bytesPerRec;
        # read data
        fid  = open(fls[k],'rb')
        ldata= np.fromfile(fid, dtype='>f').reshape((int(nrecs),int(imax)))
        fid.close()
        if (k==0):
            header =  ldata[0,:]
            data   =  ldata[1:,:]
        else:
            header = np.append(header,ldata[0,:],axis=0 )
            data   = np.append(data,ldata[1:,:], axis=0 )
    
    nIter = np.size( np.where( data[:,0] == 1 ) )
    types = np.dtype([('time', prec, nIter),('x', prec, nIter), ('y', prec, nIter), 
                    ('z', prec, nIter), ('i', prec, nIter), ('j', prec, nIter), 
                    ('k', prec, nIter), ('p', prec, nIter), ('u', prec, nIter), 
                    ('v', prec, nIter), ('t', prec, nIter), ('s', prec, nIter) ])
    flt = np.empty(int(max(data[:,0])), dtype=types)

    #### sort it all out
    for k in range(int(max(data[:,0]))):
        j = np.where( data[:,0] == k+1 )
        jj = np.argsort( data[j,1] )
        flt[k]['time'] = np.squeeze(data[j[0][jj],1])
        flt[k]['x'] = data[j[0][jj],2]
        flt[k]['y']= data[j[0][jj],3]
        flt[k]['z']= data[j[0][jj],4]
        flt[k]['i']= data[j[0][jj],5]
        flt[k]['j']= data[j[0][jj],6]
        flt[k]['k']= data[j[0][jj],7]
        flt[k]['p']= data[j[0][jj],8]
        flt[k]['u']= data[j[0][jj],9]
        flt[k]['v']= data[j[0][jj],10]
        flt[k]['t']= data[j[0][jj],11]
        flt[k]['s']= data[j[0][jj],12]

    return flt, data, header